In [1]:
!conda install --yes keras


Solving environment: ...working... done

# All requested packages already installed.

Retrieving notices: ...working... done


In [28]:
!conda install --yes tensorflow


Solving environment: ...working... done

# All requested packages already installed.

Retrieving notices: ...working... done


In [29]:
from keras.preprocessing.image import ImageDataGenerator


In [30]:
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)


In [31]:
test_datagen=ImageDataGenerator(rescale=1./255)


In [32]:
x_train=train_datagen.flow_from_directory(r'C:\Users\HARIHARAN\PycharmProjects\AI Analyzer for fitness enthusiasts\TRAIN_SET',target_size=(64, 64),batch_size=5,color_mode='rgb',class_mode='sparse')


Found 2626 images belonging to 5 classes.


In [33]:
x_test=test_datagen.flow_from_directory(r'C:\Users\HARIHARAN\PycharmProjects\AI Analyzer for fitness enthusiasts\TEST_SET',target_size=(64, 64),batch_size=5,color_mode='rgb',class_mode='sparse')


Found 1055 images belonging to 5 classes.


In [34]:
print(x_train.class_indices)

{'APPLES': 0, 'BANANA': 1, 'ORANGE': 2, 'PINEAPPLE': 3, 'WATERMELON': 4}


In [35]:
print(x_test.class_indices)


{'APPLES': 0, 'BANANA': 1, 'ORANGE': 2, 'PINEAPPLE': 3, 'WATERMELON': 4}


In [36]:
from collections import Counter as c
c(x_train.labels)

Counter({0: 606, 1: 445, 2: 479, 3: 621, 4: 475})

In [37]:
import numpy as np


In [38]:
import tensorflow


In [39]:
from tensorflow.keras.models import Sequential


In [40]:
from tensorflow.keras import layers

In [41]:
from tensorflow.keras.layers import Dense,Flatten

In [42]:
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dropout

In [43]:
from keras.preprocessing.image import ImageDataGenerator

In [44]:
model=Sequential()

In [45]:
classifier=Sequential()

In [46]:
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64 ,3), activation='relu'))

In [47]:
classifier.add(MaxPooling2D(pool_size=(2, 2)))

In [48]:
classifier.add(Conv2D(32, (3, 3), activation='relu'))


In [49]:
classifier.add(Flatten())

In [50]:
classifier.add(Dense(units=128, activation='relu'))

In [51]:
classifier.add(Dense(units=5, activation='softmax'))

In [52]:
classifier.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 31, 31, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 flatten_1 (Flatten)         (None, 26912)             0         
                                                                 
 dense_2 (Dense)             (None, 128)               3444864   
                                                                 
 dense_3 (Dense)             (None, 5)                 645       
                                                      

In [54]:
classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [55]:
classifier.save('nutrition.h5')

In [56]:
from tensorflow.keras.models import load_model

In [58]:
from keras.preprocessing import image

model = load_model("nutrition.h5")

In [1]:
from flask import Flask,render_template,request

In [2]:
import os

In [3]:
import numpy as np

In [4]:
from tensorflow.keras.models import load_model

In [5]:
from tensorflow.keras.preprocessing import image

In [6]:
import requests

In [7]:
app= Flask(__name__,template_folder="templates")

In [8]:
model=load_model('nutrition.h5')

In [9]:
print("Loaded model from disk")

Loaded model from disk


In [11]:
@app.route('/')
def home():
    return render_template('homepage.html')

In [12]:
@app.route('/image1',methods=['GET','POST'])
def image1():
    return render_template("image.html")

In [14]:
@app.route('/predict',methods=['GET','POST'])
def launch():
    if request.method=='POST':
        f=request.files['file']
        basepath=os.path.dirname('__file__')
        filepath=os.path.join(basepath,"uploads",f.filename)
        f.save(filepath)
        
        img=image.load_img(filepath,target_size=(64,64))
        x=image.img_to_array(img)
        x=np.expand_dims(x,axis=0)
        
        pred=np.argmax(model.predict(x), axis=1)
        print("prediction",pred)
        index=['APPLES','BANANA','ORANGE','PINEAPPLE','WATERMELON']
        result=str(index[pred[0]])
        x=result
        print(x)
        result=nutrition(result)
        print(result)
        return render_template("0.html",showcase=(result),showcase1=(x))

In [15]:
def nutrition(index):
    url = "https://calorieninjas.p.rapidapi.com/v1/nutrition"
    querystring = {"query":index}
    headers = {
        'X-RapidAPI-Key': 'daaf576556msh5fcbc747e5cb27cp14bd10jsn07d05ab509ae',
    'X-RapidAPI-Host': 'calorieninjas.p.rapidapi.com'
    }
    response = requests.request("GET",url,headers=headers, params=querystring)
    print(response.text)
    return response.json()['items']


In [ ]:
if __name__ == "__main__":
    app.run(debug=False)